In [1]:
!pip install -q transformers peft datasets torch accelerate evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k")

# Take only 100 samples
small_dataset = dataset["train"].select(range(100))

print("Dataset size:", len(small_dataset))
print("\nSample example:\n")
print(small_dataset[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Dataset size: 100

Sample example:

{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Model loaded successfully")


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Model loaded successfully


In [4]:
def preprocess(example):
    input_text = f"Instruction: {example['instruction']}\nContext: {example['context']}"
    target_text = example["response"]

    model_inputs = tokenizer(
        input_text,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    labels = tokenizer(
        target_text,
        padding="max_length",
        truncation=True,
        max_length=128
    )

    labels_ids = [
        (token if token != tokenizer.pad_token_id else -100)
        for token in labels["input_ids"]
    ]

    model_inputs["labels"] = labels_ids
    return model_inputs

tokenized_dataset = small_dataset.map(preprocess)
tokenized_dataset.set_format(type="torch")


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [5]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()


trainable params: 344,064 || all params: 77,305,216 || trainable%: 0.4451


In [6]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results_lora",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

trainer_lora = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset
)

print("Training LoRA...")
trainer_lora.train()


Training LoRA...


Step,Training Loss
10,9.688327
20,9.693647


TrainOutput(global_step=25, training_loss=9.692365264892578, metrics={'train_runtime': 3.8414, 'train_samples_per_second': 26.032, 'train_steps_per_second': 6.508, 'total_flos': 4673686732800.0, 'train_loss': 9.692365264892578, 'epoch': 1.0})

In [7]:
from peft import IA3Config

base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

ia3_config = IA3Config(
    task_type="SEQ_2_SEQ_LM"
)

ia3_model = get_peft_model(base_model, ia3_config)
ia3_model.print_trainable_parameters()


Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


trainable params: 34,816 || all params: 76,995,968 || trainable%: 0.0452


In [8]:
training_args_ia3 = TrainingArguments(
    output_dir="./results_ia3",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

trainer_ia3 = Trainer(
    model=ia3_model,
    args=training_args_ia3,
    train_dataset=tokenized_dataset
)

print("Training IA3...")
trainer_ia3.train()


Training IA3...


Step,Training Loss
10,9.685006
20,9.698271


TrainOutput(global_step=25, training_loss=9.69331771850586, metrics={'train_runtime': 7.8349, 'train_samples_per_second': 12.763, 'train_steps_per_second': 3.191, 'total_flos': 4649936486400.0, 'train_loss': 9.69331771850586, 'epoch': 1.0})

In [9]:
def generate(model, text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

test_prompt = "Instruction: Explain what machine learning is.\nContext:"

print("\nLoRA Output:\n")
print(generate(lora_model, test_prompt))

print("\nIA3 Output:\n")
print(generate(ia3_model, test_prompt))



LoRA Output:

Inventive learning is the process of learning the material by a computer.

IA3 Output:

Machine learning is a machine used to learn and learn from.
